In [2]:
import tempfile

from monai.visualize.utils import matshow3d
from monai.data import CacheDataset, ThreadDataLoader, PersistentDataset
from monai.transforms import (
    Compose,
    LoadImaged,
    MapTransform,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    CropForegroundd,
    SpatialCropd,
    RandFlipd,
    RandRotate90d,
    RandShiftIntensityd,
    EnsureTyped,
    BorderPad,
    SaveImaged,
    Lambdad,
)

from monai.data import  decollate_batch

import matplotlib.pyplot as plt
from monai.networks.nets import SwinUNETR

from monai.losses import DiceCELoss

from monai.metrics import DiceMetric, ROCAUCMetric

from monai.transforms import AsDiscrete

from tqdm import tqdm

from nodule_transforms.LoadArterysMaskd import LoadArterysMaskd
from nodule_transforms.AddCustomerMetadatad import AddCustomerMetadatad
from nodule_transforms.CropToNoduled import CropToNoduled



import os
import torch



from data_tools.datatools import load_cancer_nodules_datalist, split_data


directory = './monai_data'
root_dir = tempfile.mkdtemp() if directory is None else directory
cache_dir = './training_cache'

CANCER_NODULES_DATASET_DIR = '/media/vlermakov/data/UCSDNodules/Metastatic/'
PRE_PROCESSED_DIRECTORY = '/home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/'


datalist = load_cancer_nodules_datalist(CANCER_NODULES_DATASET_DIR, PRE_PROCESSED_DIRECTORY)

Processing MRN 32135581 Phonetic_ID Minfirag
Processing MRN 32133264 Phonetic_ID Sopoopra
Processing MRN 32130631 Phonetic_ID Tusliblu
Processing MRN 32115674 Phonetic_ID Gunifa
Processing MRN 32113418 Phonetic_ID Jasnuflo
Processing MRN 32108403 Phonetic_ID Olcisof
Processing MRN 32108403 Phonetic_ID Jebeemu
Processing MRN 32104560 Phonetic_ID Ukebjiem
Processing MRN 32093373 Phonetic_ID Socaba
Processing MRN 32092400 Phonetic_ID Lorutink
Processing MRN 32092400 Phonetic_ID Dihudi
Processing MRN 32082883 Phonetic_ID Nidogek
Processing MRN 32080054 Phonetic_ID Agunit
Processing MRN 32075094 Phonetic_ID Bekubo
Processing MRN 32072750 Phonetic_ID Jaquesif
Processing MRN 32058379 Phonetic_ID Rakajid
Processing MRN 32047099 Phonetic_ID Mohuteg
Patch ID nan is not a string
Processing MRN 32047002 Phonetic_ID Ridine
Processing MRN 32043734 Phonetic_ID Swofeku
Processing MRN 32023702 Phonetic_ID Tuerekom
Processing MRN 31995447 Phonetic_ID Hakingques
Processing MRN 31970139 Phonetic_ID Sracaq

Processing MRN 30224087 Phonetic_ID Etolut
Processing MRN 30224087 Phonetic_ID Froluju
Processing MRN 30224087 Phonetic_ID Afomid
Processing MRN 30224087 Phonetic_ID Jeyaros
Processing MRN 30224087 Phonetic_ID Eyookir
Processing MRN 30224087 Phonetic_ID Dooflaho
Processing MRN 30224087 Phonetic_ID Ulekep
Processing MRN 30989309 Phonetic_ID Frakapbad
Processing MRN 21106539 Phonetic_ID Raswizu
Processing MRN 21106539 Phonetic_ID Sufourog
Processing MRN 31009039 Phonetic_ID Hadaseb
Processing MRN 31009039 Phonetic_ID Phukrezee
Processing MRN 22858260 Phonetic_ID Unoonur
Processing MRN 22858260 Phonetic_ID Quecuge
Processing MRN 22858260 Phonetic_ID Coofapas
Processing MRN 30814084 Phonetic_ID Cooseero
Processing MRN 30331711 Phonetic_ID Nadoji
Processing MRN 8414286 Phonetic_ID Huciku
Processing MRN 8414286 Phonetic_ID Dajica
Processing MRN 8414286 Phonetic_ID Bunique
Processing MRN 8414286 Phonetic_ID Bramanas
Processing MRN 8414286 Phonetic_ID Quuquilep
Processing MRN 17126632 Phonetic

In [3]:
processed_scans = []
unprocessed_scans = []
n_processed = 0
for p in datalist:
    if p['pre_processed']:
        n_processed += 1 
        processed_scans.append(p)
    else:
        unprocessed_scans.append(p)
print(len(unprocessed_scans))

607


In [ ]:
train_data_list, val_data_list = split_data(datalist)

In [ ]:
def get_data_list_stats(data_list):
    n_pos = 0
    n_neg = 0
    n_total = 0
    for l in data_list:
        if(l['label'] == 0):
            n_neg +=1
        elif(l['label'] == 1):
            n_pos += 1
            
        n_total += 1
        
    print(f"P:{n_pos} N:{n_neg} T:{n_total} %pos: {n_pos/n_total:.2f}")



In [ ]:
get_data_list_stats(train_data_list)

In [ ]:
get_data_list_stats(val_data_list)

In [4]:
def file_name_formatter(metadict: dict, saver) -> dict:
    return {"subject": metadict['phonetic_id'] + "__" + metadict['patch_id'], "idx": 0}
    
    
if(len(unprocessed_scans) > 0):
    pre_process_transforms = Compose(
        [
            LoadImaged(keys=["image"], ensure_channel_first=True, image_only=False),
            LoadArterysMaskd(keys=["mask"], ensure_channel_first=True),
            AddCustomerMetadatad(keys=["image","mask"],meta_keys=["mrn","phonetic_id","patch_id"]),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=-1000,
                a_max=400,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            Spacingd(
                keys=["image","mask"],
                pixdim=(0.75, 0.75, 0.75),
                mode=("bilinear","nearest"),
            ),
            CropToNoduled(keys=["image","mask"]),
            SaveImaged(keys=["image"], output_name_formatter=file_name_formatter ,output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="image", output_ext=".nii.gz", resample=False),
            SaveImaged(keys=["mask"], output_name_formatter=file_name_formatter, output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="mask", output_ext=".nii.gz", resample=False)
        ]
    )

    train_ds = CacheDataset(
        data=unprocessed_scans[1:],
        transform=pre_process_transforms,
        cache_num=len(unprocessed_scans[1:]),
        cache_rate=1.0,
        num_workers=8,
    )



Loading dataset:   0%|                                                                                                                                                                                                                                          | 0/606 [00:00<?, ?it/s]

2023-10-13 12:51:20,542 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-3/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-3_image_0.nii.gz
2023-10-13 12:51:20,681 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-3/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-3_mask_0.nii.gz
2023-10-13 12:51:28,626 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-1/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-1_image_0.nii.gz
2023-10-13 12:51:28,749 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Sopoopra__639797

Loading dataset:   0%|▋                                                                                                                                                                                                                               | 2/606 [01:21<5:45:11, 34.29s/it]

2023-10-13 12:51:59,456 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-16/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-16_image_0.nii.gz
2023-10-13 12:51:59,753 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-16/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-16_mask_0.nii.gz
2023-10-13 12:51:59,818 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7_image_0.nii.gz


Loading dataset:   1%|█▊                                                                                                                                                                                                                              | 5/606 [01:42<2:27:17, 14.70s/it]

...

Loading dataset:  96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉        | 584/606 [1:28:57<06:36, 18.03s/it]

2023-10-13 14:19:18,196 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Loomako__651b8b6d885009ff8214da9c-4b40834adb8634d0-1/Loomako__651b8b6d885009ff8214da9c-4b40834adb8634d0-1_image_0.nii.gz
2023-10-13 14:19:18,616 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Loomako__651b8b6d885009ff8214da9c-4b40834adb8634d0-1/Loomako__651b8b6d885009ff8214da9c-4b40834adb8634d0-1_mask_0.nii.gz


Loading dataset:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 589/606 [1:29:04<02:34,  9.07s/it]


RuntimeError: applying transform <monai.transforms.io.dictionary.LoadImaged object at 0x7f8dc1066610>

2023-10-13 14:19:46,561 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Labfaljon__651b8db8b8a4cf7045c06ad8-740b5d2ce0b5bdda-1/Labfaljon__651b8db8b8a4cf7045c06ad8-740b5d2ce0b5bdda-1_image_0.nii.gz
2023-10-13 14:19:46,780 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Labfaljon__651b8db8b8a4cf7045c06ad8-740b5d2ce0b5bdda-1/Labfaljon__651b8db8b8a4cf7045c06ad8-740b5d2ce0b5bdda-1_mask_0.nii.gz
2023-10-13 14:19:47,179 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Togepel__601c3933d1c6c009b6c1181f-82da709b70e22cee-2/Togepel__601c3933d1c6c009b6c1181f-82da709b70e22cee-2_image_0.nii.gz
2023-10-13 14:19:47,354 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm/Togepel__601c3

In [ ]:
fname = '/media/vlermakov/data/UCSDNodules/Metastatic/data/Sukicuk__2.25.113743471811282083585069081717620800044/2.25.113743471811282083585069081717620800044/2.25.326808598846151585390527901137079074435'

In [ ]:
import re


match = re.search(r'/(\w+__)\d+\.\d+\.\d+/\d+\.\d+\.(\d+)/\d+\.\d+\.(\d+)', fname)

if match:
    result = match.group(1) + match.group(3)
    print(result)

In [ ]:
unprocessed_scans